In [52]:
# imports
import pandas as pd
import os
import numpy as np

In [38]:
from collections import defaultdict

In [35]:
# read in data
# some of these lists are weird...and maybe shouldn't be used...
# or at least filtered by review and maybe genre to eliminate weird entries

root = './data/'
files = os.listdir(root)
dframes = [pd.read_json(root + f) for f in files]
#data = pd.read_json(fname)
#data

# The descriptions are screwed up - parts of them repeat.  I'm not sure what to do about this.  
# If first reducing to words, could live with it and use unique words.

In [39]:
genres = defaultdict(int)
for dframe in dframes:
    for l in dframe.genres:
        for li in l:
            genres[li] += 1

In [47]:
master = pd.concat(dframes)

In [55]:
np.sum(master.author.str.contains('kipling'))

0

In [61]:
from validation import top_books

In [64]:
pd.DataFrame(top_books.top_books)

,author,rank,title
0,J.R.R. Tolkien,#1,The Hobbit (Lord of the Rings)
1,Julie Danneberg,#2,First Day Jitters (Mrs. Hartwells classroom ad...
2,Antoine de Saint-Exupery,#3,The Little Prince
3,Roger Priddy,#4,First 100 Words
4,Merriam-Webster,#5,Webster's Spanish-English Dictionary for Stude...
5,Dav Pilkey,#6,Dog Man: Lord of the Fleas: From the Creator o...
6,Natasha Wing,#7,The Night Before Kindergarten
7,Julia Cook,#8,My Mouth Is a Volcano!
8,Giles Andreae,#9,Giraffes Can't Dance
9,Brighter Child,#10,Multiplication 0 to 12 Flash Cards (Brighter C...
